In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import copy

In [2]:
train = pd.read_csv("/Users/sougata-8718/Downloads/train.csv")
test = pd.read_csv("/Users/sougata-8718/Downloads/test_data.csv")


In [3]:
train = train.append(test.iloc[40], ignore_index = True).drop(['Unnamed: 0'], axis= 1)

In [4]:
def get_f1_score(predictions, labels, model):
    for average in ["micro", "weighted"]:
        f1 = sklearn.metrics.f1_score(predictions, labels, average = average)
        print("F1 score for ", model.__class__.__name__, " (average = ", average, ") - ", f1)
        pass
    print("\n")
    return f1

In [5]:
vectorizers = [CountVectorizer(), TfidfVectorizer()]
naive_bayes_models = [GaussianNB, MultinomialNB]
f1_score = sklearn.metrics.f1_score

In [6]:
for vectorizer in vectorizers:
    print("\nUsing ", vectorizer.__class__.__name__,"\n")
    
    model = SVC(C = 200)
    vectorizer.fit(train["Text"])
    X_train = vectorizer.transform(train["Text"]).toarray()
    X_test = vectorizer.transform(test["Text"]).toarray()
    
    model.fit(X_train, train["Intents"])
    predictions = model.predict(X_test)
    get_f1_score(predictions, test["Intents"], model)
    
    
    
    for naive_bayes_model in naive_bayes_models:
        model = naive_bayes_model()
        model.fit(X_train, train["Intents"])
        predictions = model.predict(X_test)
        get_f1_score(predictions, test["Intents"], model)
        
        pass
    pass



Using  CountVectorizer 

F1 score for  SVC  (average =  micro ) -  0.8292682926829268
F1 score for  SVC  (average =  weighted ) -  0.8243902439024391


F1 score for  GaussianNB  (average =  micro ) -  0.8536585365853658
F1 score for  GaussianNB  (average =  weighted ) -  0.8583042973286874


F1 score for  MultinomialNB  (average =  micro ) -  0.6097560975609756
F1 score for  MultinomialNB  (average =  weighted ) -  0.6682926829268292



Using  TfidfVectorizer 

F1 score for  SVC  (average =  micro ) -  0.9512195121951219
F1 score for  SVC  (average =  weighted ) -  0.9447154471544715


F1 score for  GaussianNB  (average =  micro ) -  0.8780487804878049
F1 score for  GaussianNB  (average =  weighted ) -  0.8940766550522649


F1 score for  MultinomialNB  (average =  micro ) -  0.3170731707317073
F1 score for  MultinomialNB  (average =  weighted ) -  0.41832395247029397




/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/py

In [7]:
# greetings = ['Greetings']
teams = ['Team_leadership_Management', 'Sub_team queries', 'Team_features queries', 'Team_members_count queries', 'Teams_count queries', 'Team_members_information queries', 'Team_members_list queries']
office = ['Dress_code_related queries', 'Office_timing']
doctor = ['Doctor_appointment queries', 'Doctor_availability queries']
dining = ['Dining/mess location related queries', 'Menu related queries']
leave = ['Leave_info', 'Apply_leave procedure']
cust_feedback = ['Complaints', 'Feedback']
misc = ['Greetings','Pitstop_related queries', 'Cab booking', 'Phone_related queries',  'Policies_related queries']

# groups = {'greetings' : greetings, 'teams' : teams, 'office' : office, 'doctor' : doctor,
#         'dining' : dining, 'leave' : leave, 'userfeedback' : cust_feedback, 'misc' : misc}
groups = {'teams' : teams, 'office' : office, 'doctor' : doctor,
        'dining' : dining, 'leave' : leave, 'userfeedback' : cust_feedback, 'misc' : misc}

intent_group_map = {}
for group, intent_list in groups.items():
    for intent in intent_list:
        intent_group_map.update({intent : group})




In [8]:
train["group"] = train["Intents"].apply(lambda x : intent_group_map.get(x))
test["group"] = test["Intents"].apply(lambda x : intent_group_map.get(x))
print("Intents sorted into groups")

Intents sorted into groups


In [9]:
print("Classification scores for groups\n\n\n")

tfidf_svc_group = None
tfidf_gnb_group = None

for vectorizer in vectorizers:
    print("\nUsing ", vectorizer.__class__.__name__,"\n\n")
    
    X_train = vectorizer.transform(train["Text"]).toarray()
    X_test = vectorizer.transform(test["Text"]).toarray()
    
    model = SVC(C = 200)
    model.fit(X_train, train["group"])
    if vectorizer.__class__.__name__ == "TfidfVectorizer" and model.__class__.__name__=="SVC":
        tfidf_svc_group = model    
    predictions = model.predict(X_test)
    
    get_f1_score(predictions, test["group"], model)
    

    
    for naive_bayes_model in naive_bayes_models:
        model = naive_bayes_model()
        model.fit(X_train, train["group"])
        if vectorizer.__class__.__name__ == "TfidfVectorizer" and model.__class__.__name__ == "GaussianNB":
            tfidf_gnb_group = model
        predictions = model.predict(X_test)
        
        get_f1_score(predictions, test["group"], model)
        pass
    pass



Classification scores for groups




Using  CountVectorizer 


F1 score for  SVC  (average =  micro ) -  0.9512195121951219
F1 score for  SVC  (average =  weighted ) -  0.9519714643786753


F1 score for  GaussianNB  (average =  micro ) -  0.926829268292683
F1 score for  GaussianNB  (average =  weighted ) -  0.9270524469950581


F1 score for  MultinomialNB  (average =  micro ) -  0.8536585365853658
F1 score for  MultinomialNB  (average =  weighted ) -  0.8567711962833913



Using  TfidfVectorizer 


F1 score for  SVC  (average =  micro ) -  0.975609756097561
F1 score for  SVC  (average =  weighted ) -  0.9767711962833915


F1 score for  GaussianNB  (average =  micro ) -  0.926829268292683
F1 score for  GaussianNB  (average =  weighted ) -  0.9270524469950581


F1 score for  MultinomialNB  (average =  micro ) -  0.6585365853658537
F1 score for  MultinomialNB  (average =  weighted ) -  0.7396135571745328




/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


(1, 187)

In [10]:
tfidf_vectorizer_groups = copy.deepcopy(vectorizers[1])


In [11]:
selected_vectorizers = vectorizers
selected_models = [GaussianNB(), SVC(C = 200)]
group_models = {}

for group, intents in groups.items():
    print("\n\nGroup - ",group)
    X_train_group = train[train["group"] == group]
    X_test_group = test[test["group"] == group]
    
    vectorizer = TfidfVectorizer()
    vectorizer.fit(X_train_group["Text"])
    models = []
    for model in selected_models:
        model.fit(vectorizer.transform(X_train_group["Text"]).toarray(), X_train_group["Intents"])
        predictions = model.predict(vectorizer.transform(X_test_group["Text"]).toarray())
        get_f1_score(predictions, X_test_group["Intents"], model)
        models.append(copy.deepcopy(model))
        pass
    if vectorizer.__class__.__name__ == "TfidfVectorizer":
        group_models.update({group : (vectorizer,models)})



Group -  teams
F1 score for  GaussianNB  (average =  micro ) -  0.9166666666666666
F1 score for  GaussianNB  (average =  weighted ) -  0.9222222222222222


F1 score for  SVC  (average =  micro ) -  0.9166666666666666
F1 score for  SVC  (average =  weighted ) -  0.8888888888888888




Group -  office
F1 score for  GaussianNB  (average =  micro ) -  1.0
F1 score for  GaussianNB  (average =  weighted ) -  1.0


F1 score for  SVC  (average =  micro ) -  1.0
F1 score for  SVC  (average =  weighted ) -  1.0




Group -  doctor
F1 score for  GaussianNB  (average =  micro ) -  1.0
F1 score for  GaussianNB  (average =  weighted ) -  1.0


F1 score for  SVC  (average =  micro ) -  1.0
F1 score for  SVC  (average =  weighted ) -  1.0




Group -  dining
F1 score for  GaussianNB  (average =  micro ) -  1.0
F1 score for  GaussianNB  (average =  weighted ) -  1.0


F1 score for  SVC  (average =  micro ) -  1.0
F1 score for  SVC  (average =  weighted ) -  1.0




Group -  leave
F1 score for  Gaussi

/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to acc

In [13]:

group_classifications_svc = []
group_classifications_gnb = []

def predict_groups(classifier, vectorizer = tfidf_vectorizer_groups, test_data = test["Text"]):
    return classifier.predict(vectorizer.transform(test_data).toarray())

for group_classifier in [tfidf_gnb_group, tfidf_svc_group]:
    if group_classifier.__class__.__name__ == "SVC":
        group_classifications_svc = predict_groups(group_classifier, vectorizer = tfidf_vectorizer_groups)
        pass
    if group_classifier.__class__.__name__ == "GaussianNB":
        group_classifications_gnb = predict_groups(group_classifier, vectorizer = tfidf_vectorizer_groups)
    
testCopy = test.copy()

testCopy["group_predictions_gnb"] = group_classifications_gnb
testCopy["group_predictions_svc"] = group_classifications_svc

intent_predictions_gnb_from_gnb = []
intent_predictions_gnb_from_svc = []
intent_predictions_svc_from_gnb = []
intent_predictions_svc_from_svc = []

for idx in range(len(testCopy)):
    group_prediction_gnb = testCopy.iloc[idx]["group_predictions_gnb"]
    group_prediction_svc = testCopy.iloc[idx]["group_predictions_svc"]
    text = testCopy.iloc[idx]["Text"]
    
    
    for group_prediction in [group_prediction_gnb, group_prediction_svc]:
        
        vectorizer, models = group_models.get(group_prediction)
        vectorized_text = vectorizer.transform([text]).toarray()
        gnb = models[0]
        svc = models[1]
        
        if group_prediction is group_prediction_gnb:
            intent_predictions_gnb_from_gnb.append(gnb.predict(vectorized_text)[0])
            intent_predictions_svc_from_gnb.append(svc.predict(vectorized_text)[0])
            pass
        if group_prediction is group_prediction_svc:
            intent_predictions_gnb_from_svc.append(gnb.predict(vectorized_text)[0])
            intent_predictions_svc_from_svc.append(svc.predict(vectorized_text)[0])
            pass
        

In [14]:
testCopy["group_gnb_intent_gnb"] = intent_predictions_gnb_from_gnb
testCopy["group_gnb_intent_svc"] = intent_predictions_svc_from_gnb
testCopy["group_svc_intent_gnb"] = intent_predictions_gnb_from_svc
testCopy["group_svc_intent_svc"] = intent_predictions_svc_from_svc

In [15]:
for column in ["group_gnb_intent_gnb", "group_gnb_intent_svc", "group_svc_intent_gnb", "group_svc_intent_svc"]:
    preds = testCopy[column]
    print('\n\n',column,'\n\n')
    
    get_f1_score(preds, testCopy["Intents"], None)



 group_gnb_intent_gnb 


F1 score for  NoneType  (average =  micro ) -  0.8536585365853658
F1 score for  NoneType  (average =  weighted ) -  0.889430894308943




 group_gnb_intent_svc 


F1 score for  NoneType  (average =  micro ) -  0.8780487804878049
F1 score for  NoneType  (average =  weighted ) -  0.8894308943089432




 group_svc_intent_gnb 


F1 score for  NoneType  (average =  micro ) -  0.9024390243902439
F1 score for  NoneType  (average =  weighted ) -  0.921951219512195




 group_svc_intent_svc 


F1 score for  NoneType  (average =  micro ) -  0.9024390243902439
F1 score for  NoneType  (average =  weighted ) -  0.8991869918699189




/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [14]:
testCopy

,Intents,Text,group,group_predictions_gnb,group_predictions_svc,group_gnb_intent_gnb,group_gnb_intent_svc,group_svc_intent_gnb,group_svc_intent_svc
0,Greetings,Hi,misc,dining,misc,Menu related queries,Menu related queries,Greetings,Greetings
1,Greetings,Hello,misc,dining,misc,Menu related queries,Menu related queries,Greetings,Greetings
2,Team_leadership_Management,Team lead of team xyz?,teams,teams,teams,Team_leadership_Management,Team_leadership_Management,Team_leadership_Management,Team_leadership_Management
3,Team_leadership_Management,Member leadership staff of the team xyz?,teams,teams,teams,Team_leadership_Management,Team_leadership_Management,Team_leadership_Management,Team_leadership_Management
4,Office_timing,Office timings?,office,office,office,Office_timing,Office_timing,Office_timing,Office_timing
5,Office_timing,Do we follow a office timing here?,office,office,office,Office_timing,Office_timing,Office_timing,Office_timing
6,Sub_team queries,What all sub teams exist in team xyz?,teams,teams,teams,Sub_team queries,Sub_team queries,Sub_team queries,Sub_team queries
7,Sub_team queries,Sub teams of team xyz?,teams,teams,teams,Sub_team queries,Sub_team queries,Sub_team queries,Sub_team queries
8,Team_features queries,What are the current features my team working ...,teams,teams,teams,Team_features queries,Team_members_list queries,Team_features queries,Team_members_list queries
9,Team_features queries,What are the features completed by team xyz?,teams,teams,teams,Team_features queries,Team_features queries,Team_features queries,Team_features queries
